I worked with this notbeook inside the development data folder, so if you would like to run the code, download it and put it in the right folder (or change the paths in the notebook).

In [238]:
import pprint
import json
import codecs
from nltk.tree import Tree
from collections import defaultdict

In [239]:
# Specify path to data to be analyzed (I entered the path to the tutorial data for now, so that the code runs)

with codecs.open ('relations.json', 'r', encoding = 'utf-8') as pdtb_file:
    

    # Assign all relations (a list) to a variable
    relations = [json.loads(x) for x in pdtb_file];

In [240]:
# Loop through relations and select only explicit relations:

# List for explicit relations:

relations_explicit = []

for relation in relations:
    if relation['Type'] == 'Explicit':
        relations_explicit.append(relation)
        
print(len(relations_explicit))
pprint.pprint(relations_explicit[0])

14722
{'Arg1': {'CharacterSpanList': [[825, 961]],
          'RawText': 'The new "social choice" fund will shun securities of '
                     'companies linked to South Africa, nuclear power and in '
                     'some cases, Northern Ireland',
          'TokenList': [[825, 828, 145, 6, 0],
                        [829, 832, 146, 6, 1],
                        [833, 834, 147, 6, 2],
                        [834, 840, 148, 6, 3],
                        [841, 847, 149, 6, 4],
                        [847, 848, 150, 6, 5],
                        [849, 853, 151, 6, 6],
                        [854, 858, 152, 6, 7],
                        [859, 863, 153, 6, 8],
                        [864, 874, 154, 6, 9],
                        [875, 877, 155, 6, 10],
                        [878, 887, 156, 6, 11],
                        [888, 894, 157, 6, 12],
                        [895, 897, 158, 6, 13],
                        [898, 903, 159, 6, 14],
                        [904, 

In [241]:
# Split in two lists according to arg1 in same sentence and arg1 in previous sentence:

relations_ss = []
relations_ps = []
relations_other = []

for relation in relations_explicit:
    sentence_id_arg1 = relation['Arg1']['TokenList'][0][3]
    sentence_id_connective = relation['Connective']['TokenList'][0][3]
    sentence_id_arg2 = relation['Arg2']['TokenList'][0][3]
    
    if sentence_id_arg1 == sentence_id_connective == sentence_id_arg2:
        relations_ss.append(relation)
    elif int(sentence_id_arg1) == int(sentence_id_connective) - 1 == int(sentence_id_arg2) -1:
        relations_ps.append(relation)
    else: 
        relations_other.append(relation)
print(len(relations_ps))
print(len(relations_ss))

4119
8893


In [242]:
# Open parse file:

with codecs.open('parses.json', 'r', encoding = 'utf8') as parse_file:
    parses = json.load(parse_file)

In [243]:


def normalize_str(text):
    """"""
    
    text = text.lstrip('(')
    
    text = text.rstrip(')')
    text_list = text.split()
    
    clean_text_list = []
    
    for word in text_list:
        word = word.strip()
        clean_text_list.append(word)
    
    clean_text = ' '.join(clean_text_list)
    
    return(clean_text)

my_rel = relations_ss[7]

arg1_str = my_rel['Arg1']['RawText']
print(arg1_str)
print(normalize_str(arg1_str))


and that it was his mother's birthday
and that it was his mother's birthday
rtc's! Work didn t doesn t $50


In [244]:
def get_subtrees(parsed_sentence, const_type = 0):
    """"""
    tree_nltk = Tree.fromstring(parsed_sentence)
    
    tree_height = tree_nltk.height()
    
        
    const_tuple_list = []

    for s in tree_nltk.subtrees():

        label = s.label()
        
        if (label != '``') and (label != ''):
            
            subtree_tokens_str = str(s.flatten())
            #print(type(subtree_tokens_str))
            clean_subtree_str = normalize_str(subtree_tokens_str).lstrip(label).lstrip(' ')



            if const_type != 0:
                if label == const_type:
                    const_tuple_list.append((label, clean_subtree_str))

            else:

                const_tuple_list.append((label, clean_subtree_str))



    return const_tuple_list

doc_id = my_rel['DocID']
sentence_id_arg1 = my_rel['Arg1']['TokenList'][0][3]
parsed_sentence = parses[doc_id]['sentences'][sentence_id_arg1]['parsetree']  

consts = get_subtrees(parsed_sentence)

#pprint.pprint(consts)

In [379]:
def get_data(relation, arg):
    
    """returns DocID, sentenceID, Argument raw text and sentence_token_id in relations.json"""
    
    relation_id = relation['ID']
    doc_id = relation['DocID']
    sentence_id_arg = relation[arg]['TokenList'][0][3]
    arg_str = relation[arg]['RawText']   
    
    sentence_token_id_relations = []
    
    relations_token_list = relation[arg]['TokenList']
    
    for line in relations_token_list:
        sentence_token_id = line[4]
        sentence_token_id_relations.append(sentence_token_id)
    
    return relation_id, doc_id, sentence_id_arg, arg_str, sentence_token_id_relations

my_rel = relations_ss[3]
my_relid, my_docid, my_sid, my_arg, my_st = get_data(my_rel, 'Arg1')


print(get_data(my_rel, 'Arg1'))

(3186, 'wsj_0207', 1, 'He commissions and splendidly interprets fearsome contemporary scores and does some conducting', [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])


In [381]:
def parse_sentence_token_id(doc_id, sentence_id_arg, sentence_token_id_relations):
    """"""
    
    argument_id_tuples_list = []
    
    parsed_words_list = parses[doc_id]['sentences'][sentence_id_arg]['words']
    
    for number, word in enumerate(parsed_words_list):
        
        if number in sentence_token_id_relations:
        
            argument_id_tuples_list.append((number, word[0]))
        
        
    return argument_id_tuples_list

my_tuple_list = parse_sentence_token_id(my_docid, my_sid, my_st)
print(my_tuple_list)
# Looks good, if you compare these ids to the sentence_token_ids taken from relations.json. 
# Covers the span of the argument

[(13, 'He'), (14, 'commissions'), (15, 'and'), (16, 'splendidly'), (17, 'interprets'), (18, 'fearsome'), (19, 'contemporary'), (20, 'scores'), (21, 'and'), (22, 'does'), (23, 'some'), (24, 'conducting')]


In [382]:
def get_arg_list(argument_tuples_list):
    
    """"""


    # check if argument is interrupted by comparing to the sentence token ids in the relations.json file
    slice_index = None
    
    for index, pair in enumerate(argument_tuples_list):
        if pair == argument_tuples_list[-1]:
            break

        next_pair = argument_tuples_list[index + 1]


        if (next_pair[0] - pair[0]) > 1:
            slice_index = index
            
            break
            
        else:
            slice_index = None
    
    argument_tokens_as_in_parsing = []
    
    for st_id, token in argument_tuples_list:
        
        argument_tokens_as_in_parsing.append(token)  
    # Get the argument as either 1 string in a list (if it is not interrupted) or as 2 strings in a list
    # if it is interrupted

    if slice_index:
        arg1_1 = ' '.join(argument_tokens_as_in_parsing[:slice_index + 1])

        arg1_2 = ' '.join(argument_tokens_as_in_parsing[slice_index + 1:])

        arg_str_list = [arg1_1, arg1_2]
    else:

        arg_str_list = [' '.join(argument_tokens_as_in_parsing)]
    
    
    return arg_str_list

get_arg_list(my_tuple_list)

['He commissions and splendidly interprets fearsome contemporary scores and does some conducting']

In [438]:
# Map arguments to constituents

def constituent_structure(relation, arg):
    """"""

    # Dicts with argument as key and constituent as value (tuple: (label, constituent))
    arg_is_const = defaultdict(list)
    const_part_arg = defaultdict(list)
    
    
    
    rel_id, doc_id, sentence_id_arg, arg_str, sentence_token_id_relations = get_data(relation, arg)
    
    arg_tuples_list = my_tuple_list = parse_sentence_token_id(doc_id, sentence_id_arg, sentence_token_id_relations)
    
    arg_list = get_arg_list(arg_tuples_list)


    sentence = parses[doc_id]['sentences'][sentence_id_arg]['parsetree']

    const_list = get_subtrees(sentence)


    for argument in arg_list:
        argument_list = argument.split()



        for constituent in const_list:

            label, const = constituent
            c_list = const.split()
    



            if const == argument:

                arg_is_const[argument].append((label, const))
                
                break
                
            elif (const in argument) and (len(c_list) > 2):
                const_part_arg[argument].append((label, const))
                
            
            elif (const in argument) and (len(c_list) <= 2):
                if const in argument_list:
                    const_part_arg[argument].append((label, const))
                
    return  arg_is_const, const_part_arg, rel_id, doc_id



In [440]:
# Test

my_rel = relations_ss[0]

dict1, dict2, id1, id2 = constituent_structure(my_rel, 'Arg1')


print('dict1:',dict1)
print('dict2:',dict2)

if dict2:
    print('yes')

dict1: defaultdict(<class 'list'>, {})
dict2: defaultdict(<class 'list'>, {'or receive cash from the funds': [('IN', 'from'), ('DT', 'the'), ('NNS', 'funds'), ('NNS', 'funds'), ('CC', 'or'), ('VP', 'receive cash from the funds'), ('VB', 'receive'), ('NP', 'cash from the funds'), ('NP', 'cash'), ('NN', 'cash'), ('PP', 'from the funds'), ('IN', 'from'), ('DT', 'the'), ('NNS', 'funds')]})
yes


In [444]:
# WE have two dictionaries now, one containing arguments that match constituents and one of arguments that do 
# not match constituents. For the latter, we want to get the constituents that make up the argument.

def argument_structure(relation, arg):
    """"""
   
        

    arg_is_const_dict, dict4, rel_id, doc_id = constituent_structure(relation, arg)


    # For all other cases, find the biggest constituents that make up the argument: 
    arg_const_dict = defaultdict(list)

    # List of all arguments in the dictionary (interrupted arguments are two seperate keys)
    # Make a list of all keys
    arg = list(dict4.keys())


    # for some reason, there was an empty list in one case (which makes the program break down)
    if arg:
    
        # The argument is in a list now, but we need the string (there can only be one item in the list)
        arg = arg[0]

        # Print the argument:
        print('\n\nthe argument: ',arg)

        # Make a seperate list of all constituents (the values of each argument in the original dict)
        const_list = list(dict4.values())
        

        # Print arguments to look at them:
        #pprint.pprint(const_list)

        # Make an empty list that will contain the lengths of constituents, the constituents and their label
        const_len_list = []
        # Again ,the constituents are lists, but there can only be one constituent per list (const_list[0])
        for const in const_list[0]:
            # the dict values were tuples of the constituent label and the constituent, unpack them:
            label, c = const
            
            # In order to get the lenght of the constituent, make a list:
            c_list = c.split()
            # Fill the list with the constituent length, the constituent and its label
            const_len_list.append((len(c_list), c, label))

        # Sort the list so the longest constituents will be on top
        sorted_const_len_list = sorted(const_len_list, reverse = True)

        # Start a list of the longest constituets (there might be two of the same length or the 
        # same analyzed as two different kinds of phrases (e.g. sentence and verb phrase))
        longest_consts = []

        # Append the longest ones to the list:
        for const in sorted_const_len_list:
            # Unpack tuple in length, constituent and label:
            length, c, l = const

            # Same constituents analyzed as different phrase structures should be appended:
            if (sorted_const_len_list[0][0] == length) and (sorted_const_len_list[0][1] == c) :

                longest_consts.append(const)


        # Print the list to see how many there are:
        print('the longest constituent: ',longest_consts)
        
        

        # Add longest constituent to arg_const_dict:
        arg_const_dict[arg].append((longest_consts[0][2],longest_consts[0][1]) )

        # Remove the longest constituents from the list of constituents that make up the argument 
        # (simply take the first one, as the label doesn't matter here)

        rest = arg.replace(longest_consts[0][1], '')

        # Print the rest of the argument:
        print('my rest', rest)


        # Loop through all constituents that make up the argument (sorted according to size):
        for const in sorted_const_len_list:
            # Position 0 is length, position 1 is the constituent and 2 the label
            l = const[2] # label
            c = const[1].strip() # constituent string
            rest = rest.strip()


            # If the constituent matches the rest of the agument, we have found the two constituents 
            # making up the argument and can leave the loop)
            if c == rest:
                print('constituent matches rest: ',l, c)
                arg_const_dict[arg].append((l, c))
                break

            # If this is not the case, but the constituent is part of the rest of the argument, 
            # we have found another constituent and want to remove it from the rest. Replacing it with
            # an empty character is a problem, because if we have a constituent like 'a', all as will 
            # be removed from the string (resulting in funny rest strings like 'tody' instead of today). 
            # Therefore, I use two characters as a limit for replacing characters with ''. In cases in 
            # which constituents are only two characters long, I first split up the rest and make sure
            # to only exclude the entire token (and not characters within tokens). Then I join the rest 
            # again. 

            elif c in rest:

                rest_list = rest.split()



                if len(c) > 2:

                    rest = rest.replace(c, '').strip()
                    print('another constituent', l, c)
                    arg_const_dict[arg].append((l, c))

                elif (len(c) <= 2) and (c in rest_list):
                    rest_list.remove(c)
                    rest = ' '.join(rest_list)
                    print('another constituent', l, c)
                    arg_const_dict[arg].append((l,c))

    return arg_is_const_dict, arg_const_dict, rel_id

In [448]:
# Test:
my_rel = relations_ss[3]
argument_structure(my_rel, 'Arg1')
#argument_structure(my_rel, 'Arg2')

(defaultdict(list,
             {'He commissions and splendidly interprets fearsome contemporary scores and does some conducting': [('S',
                'He commissions and splendidly interprets fearsome contemporary scores and does some conducting')]}),
 defaultdict(list, {}),
 3186)

In [451]:
for rel in relations_ss[:5]:
    #print(n)
    dict1, dict2, rel_id = argument_structure(rel, 'Arg1')
    
    
    
   
 



the argument:  or receive cash from the funds
the longest constituent:  [(5, 'receive cash from the funds', 'VP')]
my rest or 
constituent matches rest:  CC or


the argument:  Solo woodwind players have to be creative
the longest constituent:  [(3, 'Solo woodwind players', 'NP')]
my rest  have to be creative
another constituent TO to
another constituent VBP have
another constituent JJ creative
constituent matches rest:  VB be


the argument:  Solo woodwind players have to be creative if they want to work a lot
the longest constituent:  [(10, 'to be creative if they want to work a lot', 'VP'), (10, 'to be creative if they want to work a lot', 'S')]
my rest Solo woodwind players have 
another constituent NP Solo woodwind players
constituent matches rest:  VBP have


the argument:  Today , the pixie-like clarinetist has mostly dropped the missionary work
the longest constituent:  [(3, 'the pixie-like clarinetist', 'NP')]
my rest Today ,  has mostly dropped the missionary work
another c